In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.express as px
px.defaults.template =  "plotly_white"

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT
from tqdm import tqdm


import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [ ]:

# filenames = glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_[269].root") 
# filenames = filenames + glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_1[28].root") 
# filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_[269].root")
# filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_1[28].root")
# filenames = glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_[9].root") 
# filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_[9].root")


filenames = glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[12345678]*-TPCMON_*.root") 
filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_*.root")

filenames.sort()
pprint.pprint(filenames)

In [3]:
%%cpp -d

#include<sPhenixStyle.C>

In [4]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [5]:
%jsroot on

In [6]:

pedestal_sample_time = [20, 350]
drift_sample_time = [100, 250]
sample_width = 5


expo = ROOT.TF1("expo_pedestal", "[0] + [1]*exp([2]*x)", 60, 1000)

In [7]:
file_tmp = ROOT.TFile.Open(filenames[50])


c1 = ROOT.TCanvas( 'QA', 'QA', 1000, 500)

hist = file_tmp.Get("COUNTS_vs_SAMPLE_1D_R3")
expo.SetParameters(hist.GetBinContent(2), hist.GetMaximum(), -0.01)
hist.Fit("expo_pedestal", "RMS0", "", 60, 300)
hist.DrawClone()
expo.DrawClone("same")
print(f"decay constant: {-1/expo.GetParameter(2)}")
c1.Draw("")



decay constant: 173.8508464572629
 FCN=1456.75 FROM HESSE     STATUS=OK             16 CALLS         162 TOTAL
                     EDM=8.19155e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.58902e+03   1.24012e+02   3.15177e-02  -7.32713e-06
   2  p1           5.78853e+04   6.37550e+01   8.99984e-02  -9.73410e-07
   3  p2          -5.75206e-03   3.49373e-05   9.19417e-09  -2.73362e+01


In [ ]:

runnumber_list = []
sector_list = []
decay_constant_list = []
decay_constant_err_list = []

first_run = True

for filename in tqdm(filenames):
    print(f"Openning filename ... {filename}")
    file = ROOT.TFile.Open(filename)

    runnumber = int(filename.split("-")[-2].split("_")[-1])
    sector = int(filename.split(".")[-2].split("_")[-1])

    if runnumber == 54918 or runnumber == 54919:
        continue

    list = file.GetListOfKeys()

    first_hist = True

    for key in list:
        keyname = key.GetName()
        if not re.match("COUNTS_vs_SAMPLE_1D_R*", keyname):
            continue

        hist = file.Get(keyname)
        if first_hist:
            hist_sum = hist.Clone(f"h_{runnumber}_{sector}")
            first_hist = False
        else:
            hist_sum.Add(hist)
        
        print(f"{runnumber}/Sector{sector}/{keyname}:  statistics {hist_sum.GetSum()}")
    
    if hist_sum.GetSum() < 1000000:
        print(f"Skipping run {runnumber}/Sector{sector} due to low statistics {hist_sum.GetSum()}")
        continue

    if not first_hist:
        expo.SetParameters(hist_sum.GetBinContent(2), hist_sum.GetMaximum(), -0.01)
        hist_sum.Fit("expo_pedestal", "RMS0", "", 60, 300)
        decay_constant = -1/expo.GetParameter(2)
        if abs(expo.GetParError(2)/expo.GetParameter(2)) > 0.1:
            print(f"Skipping run {runnumber}/Sector{sector} due to large error {abs(expo.GetParError(2)/expo.GetParameter(2))}")
            continue
        if expo.GetParameter(1) <1000:
            print(f"Skipping run {runnumber}/Sector{sector} due to low drift window {expo.GetParError(1)}")
            continue

        runnumber_list.append(runnumber)
        sector_list.append(sector)
        decay_constant_list.append( decay_constant )
        decay_constant_err_list.append(abs(expo.GetParError(2)/expo.GetParameter(2)*decay_constant))

        if first_run:
            first_run = False

    file.Close()




In [9]:
# pprint.pprint(decay_constant_dict)
df = pd.DataFrame({'Run Number': runnumber_list, 'Sector': sector_list, 
                   'Decay Constant [50ns]': decay_constant_list,
                   'Decay Constant Error': decay_constant_err_list})
print (df)

      Run Number  Sector  Decay Constant [50ns]  Decay Constant Error
0          54873       0             120.446273              0.101338
1          54873       1             120.306288              0.096627
2          54873      10             124.375325              0.098244
3          54873      11             115.549941              0.089245
4          54873      12             129.144923              0.097506
...          ...     ...                    ...                   ...
2788       54897       5             127.947083              0.205795
2789       54897       6             144.152971              0.224750
2790       54897       7             125.783626              0.188858
2791       54897       8             130.423975              0.209489
2792       54897       9             140.822179              0.231564

[2793 rows x 4 columns]


In [10]:

fig = px.scatter(df, x="Run Number", y="Decay Constant [50ns]", error_y="Decay Constant Error", 
                 color="Sector", labels={"Sector": "Sector"},
                 title="<b>sPHENIX</b> Internal, Run24 AuAu, TPC [ADC Hit vs drift time]'s Exponential Decay Constant"
                 , height=800)
fig.update_xaxes(tickformat="d")
fig.update_layout(yaxis=dict(range=[0, 300]))


fig.add_trace(go.Scatter(
    x=[54882, 54882, 54882],
    y=[0, 250, 300],
    mode="lines+text",
    name="Gas change", showlegend = False,
    text=["", "New Iso Bottle", ""],
    textposition="middle right"
))

fig.add_annotation(
    x=54860,
    y=300,
    text="Trigger latency test",
    showarrow=True,
    xanchor="auto",
)

fig.show()